In [4]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [5]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark:   0%|          | 0/10000 [00:00<?, ?it/s]

benchmark: 100%|██████████| 10000/10000 [00:09<00:00, 1101.65it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:377: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



{'basic': {'expected_sample_burst_size': 2.9977945131791284,
           'expected_sample_data_size': 16.419185942262864,
           'expected_time_to_recover': 99.06480000000003,
           'packet_size': 5,
           'payload_bits': 80.0,
           'std_time_to_recover': 44.9431173925441},
 'derived': {'bit_efficency': 0.19154038847309016,
             'bit_efficency_ratio': 0.16151044568807482,
             'bit_efficency_sigma': 0.08229969181327546,
             'bit_redundancy': 415.3240000000002,
             'ideal_bit_efficency': 0.19099882164090226,
             'ideal_bit_redundancy': 338.85075160520284,
             'ideal_packet_efficiency': 0.9549941082045112,
             'ideal_packet_redundancy': 67.77015032104057,
             'ideal_time_to_recover': 83.77015032104057,
             'ideal_transmitted_bits': 418.85075160520284,
             'packet_efficiency': 0.8075522284403741,
             'packet_redundancy': 83.06480000000003,
             'permeability': 0.8456

benchmark: 100%|██████████| 10000/10000 [00:05<00:00, 1840.76it/s]


...

{'basic': {'expected_sample_burst_size': 2.9966362107368516,
           'expected_sample_data_size': 16.414291558707212,
           'expected_time_to_recover': 32.551799999999986,
           'packet_size': 5,
           'payload_bits': 25.0,
           'std_time_to_recover': 45.58020092935089},
 'derived': {'bit_efficency': 0.3045708014658188,
             'bit_efficency_ratio': 0.15360133694603684,
             'bit_efficency_sigma': 0.1803746385647343,
             'bit_redundancy': 137.75899999999993,
             'ideal_bit_efficency': 0.18164320074892026,
             'ideal_bit_redundancy': 112.63245690961327,
             'ideal_packet_efficiency': 0.9082160037446013,
             'ideal_packet_redundancy': 22.526491381922654,
             'ideal_time_to_recover': 27.526491381922654,
             'ideal_transmitted_bits': 137.63245690961327,
             'packet_efficiency': 0.7680066847301842,
             'packet_redundancy': 27.551799999999986,
             'permeability': 0.

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3465.40it/s]


{'basic': {'expected_sample_burst_size': 3.0138172000317636,
           'expected_sample_data_size': 16.5230683713174,
           'expected_time_to_recover': 13.349199999999998,
           'packet_size': 5,
           'payload_bits': 23.253404328774153,
           'std_time_to_recover': 5.386358636407345},
 'derived': {'bit_efficency': 0.4004335497351568,
             'bit_efficency_ratio': 0.34838648501444514,
             'bit_efficency_sigma': 0.1391843786462344,
             'bit_redundancy': 43.49259567122584,
             'ideal_bit_efficency': 0.4119323808008358,
             'ideal_bit_redundancy': 33.19616218398049,
             'ideal_packet_efficiency': 2.059661904004179,
             'ideal_packet_redundancy': 6.639232436796098,
             'ideal_time_to_recover': 11.289913302550929,
             'ideal_transmitted_bits': 56.44956651275464,
             'packet_efficiency': 1.7419324250722257,
             'packet_redundancy': 8.698519134245167,
             'permeability

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3423.22it/s]


{'basic': {'expected_sample_burst_size': 3.0153205531112506,
           'expected_sample_data_size': 16.404776869893148,
           'expected_time_to_recover': 13.9244,
           'packet_size': 5,
           'payload_bits': 23.253404328774153,
           'std_time_to_recover': 4.299916817800084},
 'derived': {'bit_efficency': 0.3598354909939128,
             'bit_efficency_ratio': 0.33399506375533816,
             'bit_efficency_sigma': 0.08891186477385693,
             'bit_redundancy': 46.36859567122585,
             'ideal_bit_efficency': 0.39538585183898817,
             'ideal_bit_redundancy': 35.55852386900964,
             'ideal_packet_efficiency': 1.976929259194941,
             'ideal_packet_redundancy': 7.111704773801927,
             'ideal_time_to_recover': 11.762385639556758,
             'ideal_transmitted_bits': 58.81192819778379,
             'packet_efficiency': 1.6699753187766908,
             'packet_redundancy': 9.27371913424517,
             'permeability': 0.844

### LT vs ANFEXT2L benchmark sweep
This section benchmarks LT and `anfext2l_undegenerate` across multiple payload sizes, caches the distributions, and plots the expected time-to-recover (TTR) and bit efficiency per payload.

In [6]:
from __future__ import annotations

import importlib
import json
from datetime import datetime
from pathlib import Path
import math
import numpy as np

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from tqdm.auto import tqdm

import impls.anfext2l_undegenerate as anfext2l_undegenerate
import impls.lt as lt
from benchmark_utils import benchmark, compute_distribution_stats

# Always reload to pick up local code edits when rerunning the cell
importlib.reload(anfext2l_undegenerate)
importlib.reload(lt)

# -----------------------------------------------------------------------------
# Configuration (adjust N or payload grids if you want to re-run with new params)
# -----------------------------------------------------------------------------
N = 5


def _payload_sizes_for_anfext2l(
    n: int, min_bits: int, max_bits: int, max_payload: int | None = None
) -> list[int]:
    base = (1 << n) - 2
    sizes: list[int] = []
    m = 1
    while True:
        payload = base**m
        bits = math.log2(payload)
        if bits > max_bits or (max_payload is not None and payload > max_payload):
            break
        if bits >= min_bits:
            sizes.append(payload)
        m += 1
    return sizes


ANF_OVERRIDE_FN = getattr(anfext2l_undegenerate, "override_D", None)
ANF_MAX_PAYLOAD = ANF_OVERRIDE_FN(N) if ANF_OVERRIDE_FN is not None else None
LT_OVERRIDE_FN = getattr(lt, "override_D", None)

PAYLOAD_BITS_BY_SCHEME = {
    "lt": [1 << bits for bits in (5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)],
    "anfext2l_undegenerate": _payload_sizes_for_anfext2l(
        N,
        min_bits=4,
        max_bits=65,
        max_payload=ANF_MAX_PAYLOAD,
    ),
}
BENCHMARK_CFG = dict(passes=200, iters_bound=400, processes=None, progress_update=100)
PLOT_WIDTH = 1050  # 1.5x wider plots
RESULTS_PATH = Path("research/lt_vs_anfext2l_results.jsonl")
RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)

SCHEMES = {
    "lt": {
        "producer": lt.producer_constructor,
        "recoverer": lt.recoverer_constructor,
        "override": LT_OVERRIDE_FN,
    },
    "anfext2l_undegenerate": {
        "producer": anfext2l_undegenerate.producer_constructor,
        "recoverer": anfext2l_undegenerate.recoverer_constructor,
        "override": ANF_OVERRIDE_FN,
    },
}


def _rgba_from_hex(hex_color: str, alpha: float) -> str:
    hex_color = hex_color.lstrip("#")
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f"rgba({r}, {g}, {b}, {alpha})"


SCHEME_COLORS = {
    "lt": "#1f77b4",
    "anfext2l_undegenerate": "#d62728",
}

SCHEME_SIGMA_FILLS = {
    scheme: {
        "1sigma": _rgba_from_hex(color, 0.35),
        "2sigma": _rgba_from_hex(color, 0.18),
    }
    for scheme, color in SCHEME_COLORS.items()
}


def _load_records(path: Path) -> list[dict]:
    if not path.exists():
        return []
    records: list[dict] = []
    with path.open("r", encoding="utf-8") as fh:
        for line in fh:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records


def _append_record(path: Path, record: dict) -> None:
    with path.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(record) + "\n")


records = _load_records(RESULTS_PATH)
existing_keys = {(rec["scheme"], rec["D"], rec["N"]) for rec in records}
plan = [
    (scheme, payload_size)
    for scheme, payloads in PAYLOAD_BITS_BY_SCHEME.items()
    for payload_size in payloads
]
pending = [
    (scheme, payload_size)
    for scheme, payload_size in plan
    if (scheme, payload_size, N) not in existing_keys
]

if pending:
    total_jobs = len(pending)
    print(f"Running {total_jobs} new benchmarks (N={N}).")
    with tqdm(total=total_jobs, desc=f"Benchmarks (N={N})") as bar:
        for scheme, payload_size in pending:
            impl = SCHEMES[scheme]
            D = payload_size
            payload_bits = math.log2(D)
            override = impl.get("override")
            if override is not None:
                max_d = override(N)
                if D > max_d:
                    raise ValueError(
                        f"Payload size D={D} (bits={payload_bits:.2f}) exceeds {scheme} support for N={N}"
                    )
            bench_result = benchmark(
                impl["producer"],
                impl["recoverer"],
                N,
                D,
                **BENCHMARK_CFG,
            )
            record = {
                "scheme": scheme,
                "payload_bits": payload_bits,
                "N": N,
                "D": D,
                "timestamp": datetime.utcnow().isoformat(),
                "benchmark_cfg": BENCHMARK_CFG,
                "result": bench_result,
            }
            records.append(record)
            _append_record(RESULTS_PATH, record)
            bar.update(1)
else:
    print(f"All requested payload benchmarks already cached for N={N}.")

records_df = pd.DataFrame(records)
records_df = records_df[records_df["N"] == N].copy()
if records_df.empty:
    raise RuntimeError(
        "No cached benchmark rows for the current N; run the cell again to generate them."
    )

summary_rows = []
for rec in records_df.to_dict("records"):
    stats = compute_distribution_stats(rec["N"], rec["D"], rec["result"])
    metrics = stats["metrics"]
    summary_rows.append(
        {
            "scheme": rec["scheme"],
            "payload_bits": rec["payload_bits"],
            "expected_ttr": metrics["basic"]["expected_time_to_recover"],
            "ttr_sigma": metrics["basic"].get("std_time_to_recover", float("nan")),
            "bit_efficiency": metrics["derived"]["bit_efficency"],
            "bit_efficiency_sigma": metrics["derived"].get(
                "bit_efficency_sigma", float("nan")
            ),
            "bit_redundancy": metrics["derived"].get("bit_redundancy", float("nan")),
            "ideal_bit_efficency": metrics["derived"].get(
                "ideal_bit_efficency", float("nan")
            ),
            "ideal_bit_redundancy": metrics["derived"].get(
                "ideal_bit_redundancy", float("nan")
            ),
            "ideal_packet_efficiency": metrics["derived"].get(
                "ideal_packet_efficiency", float("nan")
            ),
            "ideal_packet_redundancy": metrics["derived"].get(
                "ideal_packet_redundancy", float("nan")
            ),
            "ideal_time_to_recover": metrics["derived"].get(
                "ideal_time_to_recover", float("nan")
            ),
            "ideal_transmitted_bits": metrics["derived"].get(
                "ideal_transmitted_bits", float("nan")
            ),
            "packet_efficiency": metrics["derived"].get(
                "packet_efficiency", float("nan")
            ),
            "packet_redundancy": metrics["derived"].get(
                "packet_redundancy", float("nan")
            ),
        }
    )

summary_df = pd.DataFrame(summary_rows).sort_values(["scheme", "payload_bits"])
print("Cached benchmark summary:")
display(summary_df)

fig_ttr = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    expected_arr = group["expected_ttr"].to_numpy(dtype=float)
    sigma_arr = group["ttr_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = expected_arr + mult * sigma_arr
            lower = np.clip(expected_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma"
            fig_ttr.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_ttr.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["expected_ttr"],
            mode="lines+markers",
            name=f"{scheme} E[TTR]",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>E[TTR]=%{y:.2f} packets<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_ttr.update_layout(
    title=f"Expected time-to-recover vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="expected TTR (packets)",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_ttr.show()

fig_eff = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    eff_arr = group["bit_efficiency"].to_numpy(dtype=float)
    sigma_arr = group["bit_efficiency_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = eff_arr + mult * sigma_arr
            lower = np.clip(eff_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma (bit eff)"
            fig_eff.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_eff_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_eff.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["bit_efficiency"],
            mode="lines+markers",
            name=f"{scheme} bit efficiency",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>efficiency=%{y:.4f}<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_eff.update_layout(
    title=f"Bit efficiency vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="payload bits per transmitted bit",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_eff.show()

ADDITIONAL_METRICS = [
    ("bit_redundancy", "Bit redundancy vs payload size", "extra bits"),
    (
        "ideal_bit_efficency",
        "Ideal bit efficiency vs payload size",
        "ideal bits per transmitted bit",
    ),
    (
        "ideal_bit_redundancy",
        "Ideal bit redundancy vs payload size",
        "ideal extra bits",
    ),
    (
        "ideal_packet_efficiency",
        "Ideal packet efficiency vs payload size",
        "payload bits per ideal packet",
    ),
    (
        "ideal_packet_redundancy",
        "Ideal packet redundancy vs payload size",
        "ideal extra packets",
    ),
    ("ideal_time_to_recover", "Ideal time-to-recover vs payload size", "ideal packets"),
    (
        "ideal_transmitted_bits",
        "Ideal transmitted bits vs payload size",
        "ideal transmitted bits",
    ),
    (
        "packet_efficiency",
        "Packet efficiency vs payload size",
        "payload bits per packet",
    ),
    ("packet_redundancy", "Packet redundancy vs payload size", "extra packets"),
]

for metric_key, title, y_axis in ADDITIONAL_METRICS:
    if metric_key not in summary_df.columns:
        continue
    fig = go.Figure()
    for scheme, group in summary_df.groupby("scheme"):
        group = group.sort_values("payload_bits")
        color = SCHEME_COLORS.get(scheme)
        fig.add_trace(
            go.Scatter(
                x=group["payload_bits"],
                y=group[metric_key],
                mode="lines+markers",
                name=f"{scheme} {metric_key}",
                line=dict(shape="spline", color=color),
                marker=dict(color=color),
                legendgroup=scheme,
                hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>value=%{y:.4f}<extra></extra>",
                text=[scheme] * len(group),
            )
        )
    fig.update_layout(
        title=f"{title} (N={N})",
        xaxis_title="payload bits",
        yaxis_title=y_axis,
        template="plotly_white",
        width=PLOT_WIDTH,
    )
    fig.show()

Running 25 new benchmarks (N=5).


Benchmarks (N=5): 100%|██████████| 25/25 [00:47<00:00,  1.88s/it]


Cached benchmark summary:


,scheme,payload_bits,expected_ttr,ttr_sigma,bit_efficiency,bit_efficiency_sigma,bit_redundancy,ideal_bit_efficency,ideal_bit_redundancy,ideal_packet_efficiency,ideal_packet_redundancy,ideal_time_to_recover,ideal_transmitted_bits,packet_efficiency,packet_redundancy
12,anfext2l_undegenerate,4.906891,7.550,3.522428,0.156251,0.069735,32.843109,0.152392,27.292155,0.761962,5.458431,6.439809,32.199046,0.649919,6.568622
13,anfext2l_undegenerate,9.813781,10.820,5.178571,0.221164,0.099132,44.286219,0.213195,36.218073,1.065977,7.243615,9.206371,46.031855,0.907004,8.857244
14,anfext2l_undegenerate,14.720672,12.250,5.305422,0.282055,0.109649,46.529328,0.282610,37.367672,1.413049,7.473534,10.417669,52.088344,1.201687,9.305866
15,anfext2l_undegenerate,19.627562,13.500,4.648656,0.329089,0.122397,47.872438,0.345456,37.188789,1.727281,7.437758,11.363270,56.816351,1.453894,9.574488
16,anfext2l_undegenerate,24.534453,14.215,5.272454,0.390532,0.132679,46.540547,0.407690,35.644715,2.038451,7.128943,12.035833,60.179167,1.725955,9.308109
17,anfext2l_undegenerate,29.441344,17.345,6.508147,0.379771,0.120291,57.283656,0.404402,43.360813,2.022010,8.672163,14.560431,72.802157,1.697397,11.456731
18,anfext2l_undegenerate,34.348234,19.675,6.903577,0.391729,0.131350,64.026766,0.412581,48.903954,2.062903,9.780791,16.650438,83.252188,1.745781,12.805353
19,anfext2l_undegenerate,39.255125,22.725,7.585471,0.380115,0.113216,74.369875,0.410875,56.285096,2.054377,11.257019,19.108044,95.540220,1.727398,14.873975
20,anfext2l_undegenerate,44.162015,24.640,7.834564,0.395536,0.123993,79.037985,0.422317,60.408729,2.111586,12.081746,20.914149,104.570745,1.792290,15.807597
21,anfext2l_undegenerate,49.068906,27.240,8.964508,0.393700,0.114553,87.131094,0.427917,65.600233,2.139586,13.120047,22.933828,114.669139,1.801355,17.426219
